https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html

# Trivial Case: len(input) == len(output)

https://raw.githubusercontent.com/fchollet/keras/master/examples/addition_rnn.py

In [1]:
# -*- coding: utf-8 -*-
'''An implementation of sequence to sequence learning for performing addition

Input: "535+61"
Output: "596"
Padding is handled by using a repeated sentinel character (space)

Input may optionally be inverted, shown to increase performance in many tasks in:
"Learning to Execute"
http://arxiv.org/abs/1410.4615
and
"Sequence to Sequence Learning with Neural Networks"
http://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf
Theoretically it introduces shorter term dependencies between source and target.

Two digits inverted:
+ One layer LSTM (128 HN), 5k training examples = 99% train/test accuracy in 55 epochs

Three digits inverted:
+ One layer LSTM (128 HN), 50k training examples = 99% train/test accuracy in 100 epochs

Four digits inverted:
+ One layer LSTM (128 HN), 400k training examples = 99% train/test accuracy in 20 epochs

Five digits inverted:
+ One layer LSTM (128 HN), 550k training examples = 99% train/test accuracy in 30 epochs
'''

from __future__ import print_function
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range


class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one hot integer representation
    + Decode the one hot integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        """Initialize character table.

        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One hot encode given string C.

        # Arguments
            num_rows: Number of rows in the returned one hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)


class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

# Parameters for the model and dataset.
TRAINING_SIZE = 50000
DIGITS = 3
INVERT = True

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
MAXLEN = DIGITS + 1 + DIGITS

# All the numbers, plus sign and space for padding.
chars = '0123456789+ '
ctable = CharacterTable(chars)

questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789'))
                    for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    # Skip any addition questions we've already seen
    # Also skip any such that x+Y == Y+x (hence the sorting).
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    # Pad the data with spaces such that it is always MAXLEN.
    q = '{}+{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a + b)
    # Answers can be of maximum size DIGITS + 1.
    ans += ' ' * (DIGITS + 1 - len(ans))
    if INVERT:
        # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
        # space used for padding.)
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

# Try replacing GRU, or SimpleRNN.
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

print('Build model...')
model = Sequential()
# "Encode" the input sequence using an RNN, producing an output of HIDDEN_SIZE.
# Note: In a situation where your input sequences have a variable length,
# use input_shape=(None, num_feature).
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
# As the decoder RNN's input, repeatedly provide with the last hidden state of
# RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
# length of output, e.g., when DIGITS=3, max output is 999+999=1998.
model.add(layers.RepeatVector(DIGITS + 1))
# The decoder RNN could be multiple layers stacked or a single layer.
for _ in range(LAYERS):
    # By setting return_sequences to True, return not only the last output but
    # all the outputs so far in the form of (num_samples, timesteps,
    # output_dim). This is necessary as TimeDistributed in the below expects
    # the first dimension to be the timesteps.
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

# Apply a dense layer to the every temporal slice of an input. For each of step
# of the output sequence, decide which character should be chosen.
model.add(layers.TimeDistributed(layers.Dense(len(chars))))
model.add(layers.Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

# Train the model each generation and show predictions against the validation
# dataset.
for iteration in range(1, 200):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if INVERT else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)

Using TensorFlow backend.
/Users/narukawinjidtrengam/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Generating data...
Total addition questions: 50000
Vectorization...
Training Data:
(45000, 7, 12)
(45000, 4, 12)
Validation Data:
(5000, 7, 12)
(5000, 4, 12)
Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 4, 12)             1548      
_________________________________________________________________
activation_1 (Activation)    (None, 4, 12)             0         
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_____________________________

45000/45000 [==============================] - 24s 531us/step - loss: 0.3421 - acc: 0.9040 - val_loss: 0.3055 - val_acc: 0.9188
Q 686+50  T 736  ☑ 736 
Q 2+330   T 332  ☒ 333 
Q 95+36   T 131  ☑ 131 
Q 402+20  T 422  ☑ 422 
Q 45+494  T 539  ☑ 539 
Q 241+446 T 687  ☑ 687 
Q 39+600  T 639  ☒ 649 
Q 36+50   T 86   ☑ 86  
Q 663+8   T 671  ☑ 671 
Q 581+14  T 595  ☑ 595 

--------------------------------------------------
Iteration 14
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 26s 574us/step - loss: 0.2525 - acc: 0.9392 - val_loss: 0.2531 - val_acc: 0.9251
Q 48+33   T 81   ☑ 81  
Q 596+102 T 698  ☑ 698 
Q 190+81  T 271  ☒ 272 
Q 621+67  T 688  ☑ 688 
Q 15+115  T 130  ☒ 139 
Q 16+183  T 199  ☑ 199 
Q 48+733  T 781  ☑ 781 
Q 191+576 T 767  ☑ 767 
Q 438+86  T 524  ☑ 524 
Q 290+100 T 390  ☒ 391 

--------------------------------------------------
Iteration 15
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [===

45000/45000 [==============================] - 24s 536us/step - loss: 0.0346 - acc: 0.9919 - val_loss: 0.0294 - val_acc: 0.9926
Q 437+861 T 1298 ☑ 1298
Q 4+183   T 187  ☑ 187 
Q 531+78  T 609  ☑ 609 
Q 902+371 T 1273 ☑ 1273
Q 675+30  T 705  ☑ 705 
Q 85+604  T 689  ☑ 689 
Q 609+523 T 1132 ☑ 1132
Q 637+64  T 701  ☑ 701 
Q 216+22  T 238  ☑ 238 
Q 49+914  T 963  ☑ 963 

--------------------------------------------------
Iteration 28
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 26s 585us/step - loss: 0.0199 - acc: 0.9967 - val_loss: 0.0188 - val_acc: 0.9963
Q 228+60  T 288  ☑ 288 
Q 23+453  T 476  ☑ 476 
Q 466+981 T 1447 ☑ 1447
Q 88+589  T 677  ☑ 677 
Q 25+839  T 864  ☑ 864 
Q 99+813  T 912  ☑ 912 
Q 138+823 T 961  ☑ 961 
Q 72+5    T 77   ☑ 77  
Q 429+30  T 459  ☑ 459 
Q 14+882  T 896  ☑ 896 

--------------------------------------------------
Iteration 29
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [===

45000/45000 [==============================] - 25s 558us/step - loss: 0.0078 - acc: 0.9986 - val_loss: 0.0158 - val_acc: 0.9953
Q 228+60  T 288  ☑ 288 
Q 3+267   T 270  ☑ 270 
Q 437+170 T 607  ☑ 607 
Q 402+56  T 458  ☑ 458 
Q 295+7   T 302  ☑ 302 
Q 759+36  T 795  ☑ 795 
Q 827+723 T 1550 ☑ 1550
Q 5+212   T 217  ☑ 217 
Q 9+788   T 797  ☑ 797 
Q 48+766  T 814  ☑ 814 

--------------------------------------------------
Iteration 42
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 26s 572us/step - loss: 0.0435 - acc: 0.9865 - val_loss: 0.0120 - val_acc: 0.9971
Q 551+11  T 562  ☑ 562 
Q 43+945  T 988  ☑ 988 
Q 243+12  T 255  ☑ 255 
Q 134+265 T 399  ☑ 399 
Q 29+770  T 799  ☑ 799 
Q 356+3   T 359  ☑ 359 
Q 4+712   T 716  ☑ 716 
Q 35+801  T 836  ☑ 836 
Q 940+19  T 959  ☑ 959 
Q 34+977  T 1011 ☑ 1011

--------------------------------------------------
Iteration 43
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [===

45000/45000 [==============================] - 24s 537us/step - loss: 0.0019 - acc: 1.0000 - val_loss: 0.0042 - val_acc: 0.9992
Q 76+10   T 86   ☑ 86  
Q 90+147  T 237  ☑ 237 
Q 44+208  T 252  ☑ 252 
Q 8+669   T 677  ☑ 677 
Q 197+607 T 804  ☑ 804 
Q 825+1   T 826  ☑ 826 
Q 306+795 T 1101 ☑ 1101
Q 52+328  T 380  ☑ 380 
Q 39+563  T 602  ☑ 602 
Q 13+113  T 126  ☑ 126 

--------------------------------------------------
Iteration 56
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 26s 576us/step - loss: 0.0017 - acc: 0.9999 - val_loss: 0.0036 - val_acc: 0.9993
Q 924+2   T 926  ☑ 926 
Q 562+12  T 574  ☑ 574 
Q 46+24   T 70   ☑ 70  
Q 849+866 T 1715 ☑ 1715
Q 239+0   T 239  ☑ 239 
Q 11+568  T 579  ☑ 579 
Q 5+438   T 443  ☑ 443 
Q 49+86   T 135  ☑ 135 
Q 708+579 T 1287 ☑ 1287
Q 1+78    T 79   ☑ 79  

--------------------------------------------------
Iteration 57
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [===

KeyboardInterrupt: 

# General Case: canonical sequence-to-sequence

simaple machine translation

https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html

https://github.com/fchollet/keras/blob/master/examples/lstm_seq2seq.py


## References
- Sequence to Sequence Learning with Neural Networks
    https://arxiv.org/abs/1409.3215
- Learning Phrase Representations using
    RNN Encoder-Decoder for Statistical Machine Translation
    https://arxiv.org/abs/1406.1078

In [2]:
from __future__ import print_function
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np


In [3]:
batch_size = 64
epochs = 100
latent_dim = 256
num_samples = 10000

data_path = 'fra-eng/fra.txt'

In [5]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
lines = open(data_path).read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])


print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)


Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 93
Max sequence length for inputs: 16
Max sequence length for outputs: 59


In [6]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)]
)
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)]
)
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')

decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')


In [7]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.
        # decoder_target_data ahead of decoder_input by 1 step
        if t > 0:
            decoder_target_data[i, t-1, target_token_index[char]] = 1
            
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, num_decoder_tokens))

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                    initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [8]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [9]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
         batch_size=batch_size,
         epochs=epochs,
         validation_split=0.2)

model.save('s2s.h5')

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 58s 7ms/step - loss: 0.9307 - val_loss: 0.9735
Epoch 2/100
8000/8000 [==============================] - 62s 8ms/step - loss: 0.7378 - val_loss: 0.8054
Epoch 3/100
8000/8000 [==============================] - 56s 7ms/step - loss: 0.6252 - val_loss: 0.7103
Epoch 4/100
8000/8000 [==============================] - 56s 7ms/step - loss: 0.5659 - val_loss: 0.6632
Epoch 5/100
8000/8000 [==============================] - 59s 7ms/step - loss: 0.5231 - val_loss: 0.6256
Epoch 6/100
8000/8000 [==============================] - 57s 7ms/step - loss: 0.4900 - val_loss: 0.5978
Epoch 7/100
8000/8000 [==============================] - 56s 7ms/step - loss: 0.4629 - val_loss: 0.5790
Epoch 8/100
8000/8000 [==============================] - 57s 7ms/step - loss: 0.4406 - val_loss: 0.5598
Epoch 9/100
8000/8000 [==============================] - 57s 7ms/step - loss: 0.4205 - val_loss: 0.5438
Epoch 10/100
800

8000/8000 [==============================] - 67s 8ms/step - loss: 0.0742 - val_loss: 0.7052
Epoch 79/100
8000/8000 [==============================] - 62s 8ms/step - loss: 0.0730 - val_loss: 0.7028
Epoch 80/100
8000/8000 [==============================] - 60s 8ms/step - loss: 0.0720 - val_loss: 0.7109
Epoch 81/100
8000/8000 [==============================] - 60s 7ms/step - loss: 0.0703 - val_loss: 0.7173
Epoch 82/100
8000/8000 [==============================] - 63s 8ms/step - loss: 0.0697 - val_loss: 0.7137
Epoch 83/100
8000/8000 [==============================] - 63s 8ms/step - loss: 0.0685 - val_loss: 0.7240
Epoch 84/100
8000/8000 [==============================] - 63s 8ms/step - loss: 0.0676 - val_loss: 0.7218
Epoch 85/100
8000/8000 [==============================] - 66s 8ms/step - loss: 0.0669 - val_loss: 0.7304
Epoch 86/100
8000/8000 [==============================] - 63s 8ms/step - loss: 0.0659 - val_loss: 0.7291
Epoch 87/100
8000/8000 [==============================] - 64s 8ms/st

/Users/narukawinjidtrengam/anaconda3/lib/python3.6/site-packages/keras/engine/topology.py:2344: UserWarning: Layer lstm_4 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_3/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_3/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


In [13]:
# inference mode (sampling)
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)

In [14]:
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items()
)
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items()
)

In [15]:
def decoder_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence



In [19]:
for seq_index in range(100):
    # Take one sequence (part of the training test)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decoder_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Go.
Decoded sentence: Va !

-
Input sentence: Run!
Decoded sentence: Courez !

-
Input sentence: Run!
Decoded sentence: Courez !

-
Input sentence: Wow!
Decoded sentence: Ça aorre Tom.

-
Input sentence: Fire!
Decoded sentence: Au feu !

-
Input sentence: Help!
Decoded sentence: À l'aide !

-
Input sentence: Jump.
Decoded sentence: Saute.

-
Input sentence: Stop!
Decoded sentence: Stop !

-
Input sentence: Stop!
Decoded sentence: Stop !

-
Input sentence: Stop!
Decoded sentence: Stop !

-
Input sentence: Wait!
Decoded sentence: Attendez !

-
Input sentence: Wait!
Decoded sentence: Attendez !

-
Input sentence: I see.
Decoded sentence: Je les vois.

-
Input sentence: I try.
Decoded sentence: J'essaye.

-
Input sentence: I won!
Decoded sentence: J'ai gagné !

-
Input sentence: I won!
Decoded sentence: J'ai gagné !

-
Input sentence: Oh no!
Decoded sentence: Oh non !

-
Input sentence: Attack!
Decoded sentence: Attaque !

-
Input sentence: Attack!
Decoded sentence: Attaq